In [117]:
%reload_ext autoreload
%autoreload 1
import sys
sys.path.append(r'D:\\CodeRepository\\Python\\IDMSImplementation')
%aimport idms_impl.trajectory_generator
%aimport idms_impl.stop_area_mining
%aimport idms_impl.semantic_tag_conversion
%aimport idms_impl.poi

In [149]:
# """Extract user"""

import pandas as pd
import numpy as np
orig_filename = r'C:\Users\Scott\Documents\ECNU\data\20201116\9_user.csv'
df = pd.read_csv(orig_filename, encoding='gbk')
usecols = ['SUBSTR(MSISDN,1,7)', 'STAT_DATE', 'STIME', 'END_TIME', 'ZH_LABEL', 'LATITUDE', 'NUMBERITUDE', 'DURATION', 'DATA_TOTAL']
df = df[usecols]
name_mapper = {
    "SUBSTR(MSISDN,1,7)": "USER_ID",
    "NUMBERITUDE": "LONGITUDE",
    "DATA_TOTAL": "TOTAL_DATA"
}
df.rename(columns=name_mapper, inplace=True)
df = df[df['USER_ID'] == 1505184]
df.sort_values(by=['STIME'], inplace=True)
# df.to_csv("C:/Users/Scott/Documents/ECNU/data/1505184_raw_data.csv", encoding='gbk', index=False)

filt = (df.DURATION == 0) | np.isnan(df['LATITUDE']) | np.isnan(df['LONGITUDE']) | pd.isna(df['ZH_LABEL'])
df.drop(df[filt].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df['TOTAL_DATA'] = df['TOTAL_DATA'].fillna(0.)
# # df
# df = df[df['STAT_DATE'] == 20201011]
# df = df[['LONGITUDE', 'LATITUDE']]
# df.reset_index(drop=True, inplace=True)
# df.values
# df.to_json("D:/CodeRepository/map/amap/1505184_1011.json", orient='index')
# df.plot(kind='scatter', )

In [103]:
from matplotlib import pyplot as plt
df_150 = df.copy()

df_150 = df_150[df_150['STAT_DATE'] == 20201011]
df_150_coords = df_150[['LATITUDE', 'LONGITUDE']].values
for i in range(1, df_150_coords.shape[0]):
    d = get_dist(df_150_coords[i], df_150_coords[i - 1])
    print(i, d)

1 0.0
2 0.0
3 0.0
4 0.0012352292271594614
5 0.0012352292271594614
6 0.005702073100029856
7 0.00601015456541035
8 0.006008515331816911
9 0.006008515331816911
10 0.00601015456541035
11 0.00601015456541035
12 0.006008515331816911
13 0.005700556376276885
14 0.0
15 0.0
16 0.0
17 0.0
18 0.0
19 0.0
20 0.0
21 0.0
22 0.0
23 0.0
24 0.0
25 0.0
26 0.0
27 0.0
28 0.0
29 0.0
30 0.0
31 0.0
32 0.010315857355985959
33 0.010315857355985959
34 0.0038266115979747875
35 0.006495037363101883
36 0.010315857355985959
37 0.0
38 0.0
39 0.0
40 0.005702073100029856
41 0.005702073100029856
42 0.0
43 0.0
44 0.0
45 0.0
46 0.0
47 0.003276503714543029
48 0.0028740572937481636
49 0.0012352292271594614
50 0.0
51 0.0
52 1.4723357390730501e-05
53 1.4723357390730501e-05
54 0.0018058853337796466
55 0.0006546617304018966
56 0.0023895613578794055
57 0.003276503714543029
58 0.003298183610855089
59 0.0037993082387318506
60 0.008692799117023196
61 0.0049099869151723406
62 0.0
63 0.0037993082387318506
64 0.0037993082387318506
65 0

In [58]:
from sklearn.metrics.pairwise import haversine_distances
from sklearn.cluster import DBSCAN
import math

def get_dist(p1: np.ndarray, p2: np.ndarray) -> np.float64:
    p1 = p1 * math.pi / 180
    p2 = p2 * math.pi / 180
    d = haversine_distances([p1, p2])
    return d[0, 1] * 180 / math.pi

def gen_cluster_labels(coords: np.ndarray, durs: np.ndarray, eps: float, min_dur: int) -> np.ndarray:
    coords = coords * math.pi / 180
    reps = eps * math.pi / 180
    clustering = DBSCAN(eps=reps, min_samples=min_dur, algorithm='ball_tree', metric='haversine')\
        .fit(X=coords, sample_weight=durs)
    return clustering.labels_

In [150]:
# df = df[df['STAT_DATE'] == 20201011]
# df
coords = df[['LONGITUDE', 'LATITUDE']].values
durs = df['DURATION'].values

cluster_flag = gen_cluster_labels(coords=coords, durs=durs, eps=0.003, min_dur=900)

In [151]:
df['CLUSTER_ID'] = cluster_flag.reshape(cluster_flag.shape[0], 1)
df

,USER_ID,STAT_DATE,STIME,END_TIME,ZH_LABEL,LATITUDE,LONGITUDE,DURATION,TOTAL_DATA,CLUSTER_ID
0,1505184,20201010,20201010040045,20201010081633,木樨园东六期FD-3,32.014859,118.722143,15348,21653.767578,0
1,1505184,20201010,20201010081633,20201010081714,木樨园东六期FD-2,32.014859,118.722143,41,0.000000,0
2,1505184,20201010,20201010081714,20201010083406,木樨园东六期FD-3,32.014859,118.722143,1012,0.000000,0
3,1505184,20201010,20201010083521,20201010083534,建邺地税路灯站四期LF-1,32.016282,118.719879,13,0.000000,0
4,1505184,20201010,20201010083534,20201010083626,木樨园东六期FD-3,32.014859,118.722143,52,0.000000,0
...,...,...,...,...,...,...,...,...,...,...
6253,1505184,20201031,20201031221223,20201031221654,木樨园东六期FD-2,32.014859,118.722143,271,46353.588867,0
6254,1505184,20201031,20201031221654,20201031230106,木樨园东六期FD-3,32.014859,118.722143,2652,0.000000,0
6255,1505184,20201031,20201031230106,20201031230346,木樨园东六期FD-2,32.014859,118.722143,160,0.000000,0
6256,1505184,20201031,20201031230346,20201031235758,木樨园东六期FD-3,32.014859,118.722143,3252,0.000000,0


In [152]:
df = df[df['CLUSTER_ID'] != -1]
df

,USER_ID,STAT_DATE,STIME,END_TIME,ZH_LABEL,LATITUDE,LONGITUDE,DURATION,TOTAL_DATA,CLUSTER_ID
0,1505184,20201010,20201010040045,20201010081633,木樨园东六期FD-3,32.014859,118.722143,15348,21653.767578,0
1,1505184,20201010,20201010081633,20201010081714,木樨园东六期FD-2,32.014859,118.722143,41,0.000000,0
2,1505184,20201010,20201010081714,20201010083406,木樨园东六期FD-3,32.014859,118.722143,1012,0.000000,0
3,1505184,20201010,20201010083521,20201010083534,建邺地税路灯站四期LF-1,32.016282,118.719879,13,0.000000,0
4,1505184,20201010,20201010083534,20201010083626,木樨园东六期FD-3,32.014859,118.722143,52,0.000000,0
...,...,...,...,...,...,...,...,...,...,...
6253,1505184,20201031,20201031221223,20201031221654,木樨园东六期FD-2,32.014859,118.722143,271,46353.588867,0
6254,1505184,20201031,20201031221654,20201031230106,木樨园东六期FD-3,32.014859,118.722143,2652,0.000000,0
6255,1505184,20201031,20201031230106,20201031230346,木樨园东六期FD-2,32.014859,118.722143,160,0.000000,0
6256,1505184,20201031,20201031230346,20201031235758,木樨园东六期FD-3,32.014859,118.722143,3252,0.000000,0


In [153]:
df.to_csv("C:/Users/Scott/Desktop/1505184.csv", encoding='gbk', index=False)

In [154]:
c_grp = df.groupby(['CLUSTER_ID'], sort=False)
core_coords = c_grp.apply(lambda x: x[['LONGITUDE', 'LATITUDE']].mean())
# core_coords.values

def _func(x):
    # x['LONGITUDE'] = core_coords.loc[x.CLUSTER_ID, 'LONGITUDE']
    # x['LATITUDE'] = core_coords.loc[x.CLUSTER_ID, 'LATITUDE']
    x['core_lat'] = core_coords.loc[x.CLUSTER_ID, 'LATITUDE']
    x['core_lon'] = core_coords.loc[x.CLUSTER_ID, 'LONGITUDE']
    return x

_df = df.apply(_func, axis=1)
# _df.to_csv("C:/Users/Scott/Desktop/1505184.csv", encoding='gbk', index=False)
# for cid, c in c_grp:
#     print(c[['LONGITUDE', 'LATITUDE']].mean())

In [155]:
_df[_df['STAT_DATE'] == 20201011][['core_lon', 'core_lat']].values

array([[118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
       [118.72147638,  32.00805517],
 

In [ ]:
%run ../idms_impl/poi.py
poi_gen = POIGenerator()
poi_gen.load_poi(filename=r'C:\\Users\\Scott\\Documents\\ECNU\\data\\poi\\nj_poi.csv')
poi_gen.train_data(k=10)

In [ ]:
index = poi_gen.main_model.kneighbors([[32.036952,118.784759]], return_distance=False).flatten()
poi_res = poi_gen.poi_df.loc[index]
poi_res[poi_res['大类'] == '商务住宅']

In [156]:
orig_filename = r'C:\Users\Scott\Documents\ECNU\data\20201116\9_user.csv'

usecols = ['SUBSTR(MSISDN,1,7)', 'STAT_DATE', 'STIME', 'END_TIME', 'ZH_LABEL', 'LATITUDE', 'NUMBERITUDE', 'DURATION', 'DATA_TOTAL']
name_mapper = {
    "SUBSTR(MSISDN,1,7)": "USER_ID",
    "NUMBERITUDE": "LONGITUDE",
    "DATA_TOTAL": "TOTAL_DATA"
}
%run -m idms_impl.trajectory_generator

import time

start_time = time.time()

tr_gen = TrajectoryGenerator()
tr_gen.sam.load_data(filename=orig_filename)
tr_gen.sam.format_raw_data(usecols=usecols, name_mapper=name_mapper)
tr_gen.sam.check_invalid_tr(nan_dur_theta=4*3600, dist_theta=5/111)
tr_gen.sam.gen_valid_area(point_dur_theta=20, eps=0.003, min_dur=900)
tr_gen.sam.merge_adjacent_points()
# tr_gen.stop_area_mining(filename=orig_filename,
#                         usecols=usecols,
#                         name_mapper=name_mapper,
#                         nan_dur_theta=4 * 3600,
#                         dist_theta=5 / 111,
#                         point_dur_theta=20,
#                         eps=0.0036,
#                         min_dur=900)

merge_adjacent_points


In [157]:
i_df = tr_gen.sam.df
i_df = i_df[i_df['USER_ID'] == 1505184]

In [162]:
i_df.to_csv("C:/Users/Scott/Desktop/1505184_improved.csv", encoding='gbk', index=False)

In [161]:
i_df[i_df['STAT_DATE'] == 20201011][['LONGITUDE', 'LATITUDE']].values

array([[118.72204478,  32.01562128],
       [118.694394  ,  32.016227  ],
       [118.70408245,  31.99339721],
       [118.70026273,  31.98923065],
       [118.72204478,  32.01562128]])